# Vector Space Model

## penjelasan

 **Vector Space Model (VSM)** adalah salah satu metode atau algoritma yang sering digunakan untuk sebuah sistem temu kembali informasi. Algoritma ini merupakan sebuah model yang digunakan untuk mengukur kemiripan atau kesamaan (similarity term) antar suatu dokumen dengan suatu query dengan cara pembobotan term.

Dimateri kali ini kita akan belajar membuat vektor space model dengan menggunakan data pada code sebelumnya dengan catatan hanya menampilkan sebanyak 2 kategori saja dengan jumlah masing 50 data

In [ ]:
!pip install beautifulsoup4 requests

## sebelum di dibuat vektor space model

ini adalah code kita seblumnya untuk penjelasannya bisa di lihat di slide sebelumnya di materi Crowling Web Data.

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display

# Mengambil data dari situs web
url = 'https://www.tempo.co/indeks'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('article', class_='text-card')

data = []

# Looping melalui artikel dan ekstraksi informasi
article_counter = 1
for article in articles:
    title = article.find('h2', class_='title')
    title_text = title.text.strip() if title else None

    content = article.find('p')
    content_text = content.text.strip() if content else None

    date = article.find('h4', class_='date')
    if date:
        date_text = date.text.strip()
        if "jam lalu" in date_text:
            hours_ago = int(date_text.split(' ')[0])
            publish_time = datetime.now() - timedelta(hours=hours_ago)
            date_text = publish_time.strftime('%d-%m-%Y %H:%M')
    else:
        date_text = None

    link = title.find('a')['href'] if title else None
    category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
    category = category_from_url.capitalize() if category_from_url else None

    data.append({
        'Judul Berita': title_text,
        'Isi Berita': content_text,
        'Tanggal Berita': date_text,
        'Kategori Berita': category
    })

    article_counter += 1

df = pd.DataFrame(data)

df = df.reset_index(drop=True)
df.index += 1

styled_df = df.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

# Menampilkan DataFrame dengan gaya border
display(styled_df)


,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
1,"Gerakan Tusuk 3 Paslon: Anies Sebut Hak Konstitusi, Relawan Prabowo-Gibran Bilang Rusak Demokrasi","Koordinator Nasional Prabowo-Gibran Digital Team mengatakan, Gerakan Tusuk 3 Paslon di Pilkada Jakarta berpotensi merusak demokrasi. Apa alasannya?",9 menit lalu,Nasional
2,"Jalankan Program Prabowo, Bahlil Sebut ESDM Berencana Bangun Pipa Gas dari Aceh sampai Jawa","Menurut Bahlil, Kementerian ESDM akan membangun industri Liquefied Petroleum Gas untuk di dalam negeri dengan memanfaatkan tanaman C3 dan C4",12 menit lalu,Bisnis
3,DPRD Cuma Punya Waktu 2 Hari Bahas Usulan Usulan Calon Pj GubernurJakarta,"Berdasarkan hasil rapat, tenggat waktu bagi DPRD untuk memberi usulan calon Pj Gubernur Jakarta, tetap pada Jumat, 13 September 2024.",14 menit lalu,Nasional
4,Mid-Autumn Festival di Gardens By The Bay Hadirkan Lentera yang Terinspirasi Masa Kecil,Perayaan Mid-Autumn Festival di Gardens By the Bay Festival tahun ini berlangsung mulai tanggal 4 hingga 22 September 2024,19 menit lalu,Travel
5,Thomas Djiwandono Ungkap Isi Pertemuan Prabowo dan Sri Mulyani: Tak Bahas Makan Siang Gratis,Program makan siang gratis atau program makan bergizi gratis akan diterapkan ketika mereka mulai menjabat pada Oktober mendatang.,19 menit lalu,Bisnis
6,"Dilantik Menjelang Pemerintahan Jokowi Habis, Gus Ipul: Satu Detik Saja Berharga",Gus Ipul yang baru saja dilantik sebagai Menteri Sosial menyebut setiap hitungan detik itu berarti dalam tugas barunya itu.,23 menit lalu,Nasional
7,"Soal Video Viral WNI Korban TPPO di Myanmar, Kemenlu Sudah Koordinasi dengan KBRI Yangon",Kemenlu menyatakan telah berkoordinasi dengan KBRI Yangon untuk mengevakuasi WNI korban TPPO di Myanmar.,27 menit lalu,Metro
8,"Kepala BPIP Bahas Prestasi Pribadi Saat DPR Tanya Polemik Jilbab Paskibraka, Lengkapi Kontroversi Yudian Wahyudi",Kepala BPIP Yudian Wahyudi mengungkapkan prestasi pribadinya saat DPR bertanya soal polemik jilbab Paskibraka lalu.,35 menit lalu,Nasional
9,"Gus Ipul Jabat Mensos Hanya Satu Bulan, Apa yang Mau Dilakukan?",Menteri Sosial Gus Ipul masih membutuhkan waktu sehari atau dua hari untuk memetakan pekerjaannya di kementerian.,38 menit lalu,Nasional
10,Kunci Jepang Kalahkan Timnas Bahrain 5-0 di Kualifikasi Piala Dunia 2026,Ayase Ueda dan Hidemasa Morita mencetak 2 gol sebelum Koki Ogawa melengkapi kemenangan 5-0 Jepang atas Timnas Bahrain di kualifikasi Piala Dunia 2026.,40 menit lalu,Bola


## menampilkan 2 data kategori

untuk menentukan kategori di sini saya mengambil random dari list sebanyak 2, dan membatasi untuk kategori yang di ambil sebanyak masing masing 50.

bisnis 30
Nasional 49
seleb 14
tekno 21
metro 29
dunia 27
Bola 13

In [ ]:
import random

# Mengambil data dari situs web
url = 'https://www.tempo.co/indeks'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('article', class_='text-card')

data = []

# Looping melalui artikel dan ekstraksi informasi
for article in articles:
    title = article.find('h2', class_='title')
    title_text = title.text.strip() if title else None

    content = article.find('p')
    content_text = content.text.strip() if content else None

    date = article.find('h4', class_='date')
    if date:
        date_text = date.text.strip()
        if "jam lalu" in date_text:
            hours_ago = int(date_text.split(' ')[0])
            publish_time = datetime.now() - timedelta(hours=hours_ago)
            date_text = publish_time.strftime('%d-%m-%Y %H:%M')
    else:
        date_text = None

    link = title.find('a')['href'] if title else None
    category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
    category = category_from_url.capitalize() if category_from_url else None

    data.append({
        'Judul Berita': title_text,
        'Isi Berita': content_text,
        'Tanggal Berita': date_text,
        'Kategori Berita': category
    })

df = pd.DataFrame(data)

categories = df['Kategori Berita'].unique()

# selected_categories = random.sample(list(categories), 2)
selected_categories = ['Nasional', 'Bisnis']
df_selected = df[df['Kategori Berita'].isin(selected_categories)]
df_final = df_selected.groupby('Kategori Berita').head(50)

df_final = df_final.reset_index(drop=True)
df_final.index += 1

styled_df = df_final.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

display(styled_df)


,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
1,"Gerakan Tusuk 3 Paslon: Anies Sebut Hak Konstitusi, Relawan Prabowo-Gibran Bilang Rusak Demokrasi","Koordinator Nasional Prabowo-Gibran Digital Team mengatakan, Gerakan Tusuk 3 Paslon di Pilkada Jakarta berpotensi merusak demokrasi. Apa alasannya?",17 menit lalu,Nasional
2,"Jalankan Program Prabowo, Bahlil Sebut ESDM Berencana Bangun Pipa Gas dari Aceh sampai Jawa","Menurut Bahlil, Kementerian ESDM akan membangun industri Liquefied Petroleum Gas untuk di dalam negeri dengan memanfaatkan tanaman C3 dan C4",20 menit lalu,Bisnis
3,DPRD Cuma Punya Waktu 2 Hari Bahas Usulan Usulan Calon Pj GubernurJakarta,"Berdasarkan hasil rapat, tenggat waktu bagi DPRD untuk memberi usulan calon Pj Gubernur Jakarta, tetap pada Jumat, 13 September 2024.",22 menit lalu,Nasional
4,Thomas Djiwandono Ungkap Isi Pertemuan Prabowo dan Sri Mulyani: Tak Bahas Makan Siang Gratis,Program makan siang gratis atau program makan bergizi gratis akan diterapkan ketika mereka mulai menjabat pada Oktober mendatang.,27 menit lalu,Bisnis
5,"Dilantik Menjelang Pemerintahan Jokowi Habis, Gus Ipul: Satu Detik Saja Berharga",Gus Ipul yang baru saja dilantik sebagai Menteri Sosial menyebut setiap hitungan detik itu berarti dalam tugas barunya itu.,31 menit lalu,Nasional
6,"Kepala BPIP Bahas Prestasi Pribadi Saat DPR Tanya Polemik Jilbab Paskibraka, Lengkapi Kontroversi Yudian Wahyudi",Kepala BPIP Yudian Wahyudi mengungkapkan prestasi pribadinya saat DPR bertanya soal polemik jilbab Paskibraka lalu.,43 menit lalu,Nasional
7,"Gus Ipul Jabat Mensos Hanya Satu Bulan, Apa yang Mau Dilakukan?",Menteri Sosial Gus Ipul masih membutuhkan waktu sehari atau dua hari untuk memetakan pekerjaannya di kementerian.,46 menit lalu,Nasional
8,Kemenparekraf Sebut Bakal Bantu Serap Korban PHK ke Sektor Ekonomi Kreatif,Kemenparekraf sebut sektor pariwisata dan ekonomi kreatif harus menjawab tantangan dapat menyediakan peluang usaha dan buka lapangan kerja baru,48 menit lalu,Bisnis
9,Menag Yaqut Bantah Pernah Dipanggil Pansus Haji,Menag Yaqut Cholil Qoumas mengaku tidak pernah menerima surat panggilan dari Pansus Haji.,56 menit lalu,Nasional
10,Menteri Investasi Klaim Untung Produksi Petrokimia PT Lotte Bisa Capai US$ 2 Miliar Per Tahun,Menteri Investasi sebut pembangunan proyek kawasan industri petrokimia PT Lotte Chemical Indonesia di Banten bisa menyerap 13 ribu tenaga kerja,11-09-2024 08:46,Bisnis


## setelah dibuat menjadi vektor space model

disini untuk mengubah teks pada kolom "Isi Berita" dalam DataFrame df_final menjadi representasi numerik dengan menggunakan teknik Bag of Words (BoW).Langkah pertama, CountVectorizer dari pustaka scikit-learn diinisialisasi dengan vectorizer = CountVectorizer(). Kemudian, isi berita pada kolom "Isi Berita" yang kosong diisi dengan string kosong (''), dan seluruh teks tersebut diubah menjadi representasi BoW menggunakan vectorizer.fit_transform(). Ini menghasilkan matriks dengan jumlah kata sebagai kolom dan frekuensi kata sebagai nilai.

Matriks ini kemudian diubah menjadi DataFrame baru, bow_df, dengan kolom berupa kata-kata yang ada dalam teks dan indeks yang sesuai dengan df_final. Langkah terakhir adalah menggabungkan DataFrame asli (kecuali kolom "Kategori Berita") dengan DataFrame BoW dan kolom "Kategori Berita" ditempatkan di akhir. Gabungan ini disimpan dalam df_bow menggunakan pd.concat(). Dengan demikian, DataFrame df_bow berisi representasi numerik dari teks asli beserta informasi lainnya dari df_final, kecuali kolom "Kategori Berita" yang ditempatkan di bagian akhir.

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer

# Mengambil data dari situs web
url = 'https://www.tempo.co/indeks'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('article', class_='text-card')

data = []

article_counter = 1
for article in articles:
    title = article.find('h2', class_='title')
    title_text = title.text.strip() if title else None

    content = article.find('p')
    content_text = content.text.strip() if content else None

    date = article.find('h4', class_='date')
    if date:
        date_text = date.text.strip()
        if "jam lalu" in date_text:
            hours_ago = int(date_text.split(' ')[0])
            publish_time = datetime.now() - timedelta(hours=hours_ago)
            date_text = publish_time.strftime('%d-%m-%Y %H:%M')
    else:
        date_text = None

    link = title.find('a')['href'] if title else None
    category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
    category = category_from_url.capitalize() if category_from_url else None

    data.append({
        'Judul Berita': title_text,
        'Isi Berita': content_text,
        'Tanggal Berita': date_text,
        'Kategori Berita': category
    })

    article_counter += 1


df = pd.DataFrame(data)

# Menghitung frekuensi kemunculan setiap kategori
category_counts = df['Kategori Berita'].value_counts()
top_categories = category_counts.head(2).index
df_top = df[df['Kategori Berita'].isin(top_categories)]
df_final = df_top.groupby('Kategori Berita').head(50)

df_final = df_final.reset_index(drop=True)
df_final.index += 1

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_final['Isi Berita'].fillna(''))
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=df_final.index)
df_bow = pd.concat([df_final.drop(columns='Kategori Berita'), bow_df, df_final['Kategori Berita']], axis=1)

styled_df_bow = df_bow.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

display(styled_df_bow)


Output hidden; open in https://colab.research.google.com to view.

## Mengunduh dan menambahkan stopwords

selanjutnya penabahan code yang digunakan membersihkan teks dengan menghapus stopwords dan melakukan stemming atau mengubah ke text dasar. Pertama, stopwords bahasa Indonesia diunduh dan ditambah dengan angka atau kata-kata khusus. Lalu, stemmer dari Sastrawi digunakan untuk menyederhanakan kata-kata. Fungsi clean_text memproses teks dengan menghilangkan stopwords dan menghasilkan teks yang bersih.

In [ ]:
!pip install nltk sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from nltk.corpus import stopwords as nltk_stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('indonesian'))
additional_stopwords = [
    '00', '000', '10', '16', '17', '20', '2022', '2023', '2024', '2027', '2029', '215',
    '25', '31', '36', '38', '40', '44', '522', '670', '731', '943','100',	'18',	'19',	'24',	'32',	'41',	'500'
]
stopwords.update(additional_stopwords)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming dan membersihkan teks
def clean_text(text, stopwords, stemmer):
    stemmed_text = stemmer.stem(text)
    words = stemmed_text.split()
    cleaned_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(cleaned_words)

url = 'https://www.tempo.co/indeks'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Mengambil artikel dari HTML
articles = soup.find_all('article', class_='text-card')

data = []

# Looping melalui artikel dan ekstraksi informasi
for article in articles:
    title = article.find('h2', class_='title')
    title_text = title.text.strip() if title else None

    content = article.find('p')
    content_text = content.text.strip() if content else None

    date = article.find('h4', class_='date')
    if date:
        date_text = date.text.strip()
        if "jam lalu" in date_text:
            hours_ago = int(date_text.split(' ')[0])
            publish_time = datetime.now() - timedelta(hours=hours_ago)
            date_text = publish_time.strftime('%d-%m-%Y %H:%M')
    else:
        date_text = None

    link = title.find('a')['href'] if title else None
    category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
    category = category_from_url.capitalize() if category_from_url else None

    cleaned_content = clean_text(content_text, stopwords, stemmer) if content_text else None

    data.append({
        'Judul Berita': title_text,
        'Isi Berita': cleaned_content,
        'Tanggal Berita': date_text,
        'Kategori Berita': category
    })

df = pd.DataFrame(data)

category_counts = df['Kategori Berita'].value_counts()
top_categories = category_counts.head(2).index
df_top = df[df['Kategori Berita'].isin(top_categories)]
df_final = df_top.groupby('Kategori Berita').head(50)

df_final = df_final.reset_index(drop=True)
df_final.index += 1

df_final['Isi Berita'] = df_final['Isi Berita'].apply(lambda x: clean_text(x, stopwords, stemmer))

stopwords_list = list(stopwords)

# Menyiapkan CountVectorizer untuk  (Bag of Words)
vectorizer = CountVectorizer(stop_words=stopwords_list)
X = vectorizer.fit_transform(df_final['Isi Berita'].fillna(''))
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=df_final.index)
df_bow = pd.concat([df_final.drop(columns='Kategori Berita'), bow_df, df_final['Kategori Berita']], axis=1)

styled_df_bow = df_bow.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

# Menampilkan DataFrame dengan *Bag of Words* dan gaya border
display(styled_df_bow)


Output hidden; open in https://colab.research.google.com to view.

## representasi Bag of Words menjadi representasi pembobotan TF-IDF

selanjutnya mengubah code untuk mengonversi teks berita yang sudah dibersihkan menjadi representasi TF-IDF menggunakan TfidfVectorizer dari scikit-learn. Kata-kata umum diabaikan, dan hasil TF-IDF dikonversi menjadi DataFrame. DataFrame ini digabung dengan kolom-kolom baru berisi bobot TF-IDF untuk setiap kata.

In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from nltk.corpus import stopwords as nltk_stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

# Download stopwords jika belum terunduh
nltk.download('stopwords')

stopwords = set(nltk_stopwords.words('indonesian'))

additional_stopwords = [
    '00', '000', '10', '16', '17', '20', '2022', '2023', '2024', '2027', '2029', '215',
    '25', '31', '36', '38', '40', '44', '522', '670', '731', '943', '100', '18', '19', '24', '32', '41', '500'
]
stopwords.update([str(i) for i in range(0, 1000)])  # Menghapus semua angka 0-999
stopwords.update([word.lower() for word in additional_stopwords])

# Buat objek stemmer untuk stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk membersihkan teks, melakukan stemming, menghapus stopwords dan angka
def clean_text(text, stopwords, stemmer):
    # Menghilangkan semua angka dengan regular expression
    text = re.sub(r'\d+', '', text)
    stemmed_text = stemmer.stem(text)
    words = stemmed_text.split()
    cleaned_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(cleaned_words)

# Mengambil halaman web
url = 'https://www.tempo.co/indeks'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Mengambil artikel dari HTML
articles = soup.find_all('article', class_='text-card')

data = []

# Looping melalui artikel dan ekstraksi informasi
for article in articles:
    title = article.find('h2', class_='title')
    title_text = title.text.strip() if title else None

    content = article.find('p')
    content_text = content.text.strip() if content else None

    date = article.find('h4', class_='date')
    if date:
        date_text = date.text.strip()
        if "jam lalu" in date_text:
            hours_ago = int(date_text.split(' ')[0])
            publish_time = datetime.now() - timedelta(hours=hours_ago)
            date_text = publish_time.strftime('%d-%m-%Y %H:%M')
    else:
        date_text = None

    link = title.find('a')['href'] if title else None
    category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
    category = category_from_url.capitalize() if category_from_url else None

    cleaned_content = clean_text(content_text, stopwords, stemmer) if content_text else None

    data.append({
        'Judul Berita': title_text,
        'Isi Berita': cleaned_content,
        'Tanggal Berita': date_text,
        'Kategori Berita': category
    })

# Membuat DataFrame dari data
df = pd.DataFrame(data)

# Memilih kategori berita teratas berdasarkan frekuensi
category_counts = df['Kategori Berita'].value_counts()
top_categories = category_counts.head(2).index
df_top = df[df['Kategori Berita'].isin(top_categories)]
df_final = df_top.groupby('Kategori Berita').head(50)


df_final = df_final.reset_index(drop=True)
df_final.index += 1

df_final['Isi Berita'] = df_final['Isi Berita'].apply(lambda x: clean_text(x, stopwords, stemmer))

# List stopwords untuk TfidfVectorizer
stopwords_list = list(stopwords)
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_list)
X_tfidf = tfidf_vectorizer.fit_transform(df_final['Isi Berita'].fillna(''))

# Mengonversi hasil TF-IDF menjadi DataFrame
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df_final.index)
df_tfidf = pd.concat([df_final.drop(columns='Kategori Berita'), tfidf_df, df_final['Kategori Berita']], axis=1)

styled_df_tfidf = df_tfidf.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

# Menampilkan DataFrame dengan TF-IDF
display(styled_df_tfidf)


Output hidden; open in https://colab.research.google.com to view.